Get necessary imports

In [1]:
import pandas as pd
import re
import json

Load IMDB Dataset.csv

In [2]:
df = pd.DataFrame(pd.read_csv("IMDB Dataset.csv"))

Remove all non alpha numeric characters

In [3]:
def remove_non_alphanumeric(text):
    return re.sub(r'[^a-zA-Z0-9]', ' ', text)


df['review'] = df['review'].apply(remove_non_alphanumeric)

Rename colmuns to match the openAI allowed format

In [4]:
df.rename(columns={'review': 'prompt', 'sentiment': 'completion'}, inplace=True)

Rename the positive and negative labels to 1 and 2. 1 and -1 are also a fine option, However, there where fewer bugs for the fine tuned version when using 1 and 2 (or 1 and 0 etc)

In [5]:
df['completion'] = df['completion'].replace({'positive': 1, 'negative': 2})

<ipython-input-5-7eb2dff25dec>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['completion'] = df['completion'].replace({'positive': 1, 'negative': 2})


Our dataset is extremly large. We only need a fraction to obtain significant performance increases. The % can be modified

In [6]:
# only use a % of the large dataset
dataset_length = len(df)
percentege = 0.1
dataset_length = int(dataset_length * percentege)
df = df[:dataset_length]

train_index = int(dataset_length * 0.8)
val_index = int(dataset_length * 0.9)

Save the training subsection

In [7]:
df_train = df[:train_index]
df_train.to_csv('train_data.csv', index=False)

In [8]:
df_train

,prompt,completion
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production br br The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there s a family where a little boy ...,2
4,Petter Mattei s Love in the Time of Money is...,1
...,...,...
3995,Starting off here s a synopsis Porno queen A...,1
3996,He pulled the guys guts out his butt That s a...,2
3997,This movie promised bat people It didn t deli...,2
3998,I saw the film many times and every time I am...,2


Save the validation subsection

In [9]:
df_val = df[train_index:val_index]
df_val.to_csv('val_data.csv', index=False)

In [10]:
df_val

,prompt,completion
4000,This feels as if it is a Czech version of Pear...,1
4001,When oh when will someone like Anchor Bay or...,1
4002,Just before dawn is one of the best slasher...,1
4003,A very courageous attempt to bring one of the ...,1
4004,If 1977 s Exorcist II The Heretic did him n...,2
...,...,...
4495,If I hadn t been forced to watch this for work...,2
4496,Strangeland 1998 D John Pieplow Kevin Gage...,2
4497,As long as you go into this movie with the und...,2
4498,I had a bad feeling when I saw the cheap title...,2


Save the evaluation subsection

In [11]:
df_test = df[val_index:]
df_test.to_csv('test_data.csv', index=False)

In [12]:
df_test

,prompt,completion
4500,A movie theater with a bad history of past gru...,2
4501,The first time I saw this film I wanted to li...,1
4502,I have watched some pretty poor films in the p...,2
4503,The fact that a film is on DVD doesn t guarant...,2
4504,I m not a huge Star Trek fan but I was lookin...,2
...,...,...
4995,An interesting slasher film with multiple susp...,2
4996,i watched this series when it first came out i...,1
4997,Once again Jet Li brings his charismatic prese...,1
4998,I rented this movie after hearing Chris Gore ...,2


Run this cell - here you can genereate the jsonl files. Basicaly, for the gpt fine tuned models to work, they need a specific format. They also need the files to be .jsonl

In [13]:
input_files = ['train_data.csv', 'val_data.csv']
output_files = ['train_data_prepared_gpt4.jsonl','val_data_prepared_gpt4.jsonl']

for input_file, output_file in zip(input_files, output_files):
    with open(output_file, 'w') as out_file:
        data = pd.read_csv(input_file)
        for index, row in data.iterrows():
            prompt = row["prompt"]
            completion = str(row["completion"])
            gpt4_data = {
                "messages": [
                    {"role": "system",
                     "content": "As a social scientist, your task is to analyze the sentiment of user movie reviews extracted from IMDB. Please assign a sentiment score of 1 or 2 for each movie review, where 1 signifies positive sentiment, and 2 corresponds to negative sentiment."},
                    {"role": "user", "content": prompt},
                    {"role": "assistant", "content": completion}
                ]
            }

            # Write the new data to the output file in JSONL format
            out_file.write(json.dumps(gpt4_data) + '\n')

In [17]:
input_file = 'test_data.csv'
output_file = 'test_data_prepared_gpt4.jsonl'


with open(output_file, 'w') as out_file:
      data = pd.read_csv(input_file)
      for index, row in data.iterrows():
          prompt = row["prompt"]
          gpt4_data = {
              "messages": [
                  {"role": "system",
                    "content": "As a social scientist, your task is to analyze the sentiment of user movie reviews extracted from IMDB. Please assign a sentiment score of 1 or 2 for each movie review, where 1 signifies positive sentiment, and 2 corresponds to negative sentiment."},
                  {"role": "user", "content": prompt},
              ]
          }

          # Write the new data to the output file in JSONL format
          out_file.write(json.dumps(gpt4_data) + '\n')